# **Bank Renege**
Un mostrador con un tiempo de servicio aleatorio y un cliente que reniega.

Este ejemplo modela un mostrador de un banco y sus clientes llegando a tiempos aleatorios $t$. Cada cliente tiene una cierta paciencia. Éste espera llegar al mostrador hasta que se le agote la paciencia. Si llega al mostrador, lo usa por un tiempo antes de liberarlo.

In [0]:
!pip install simpy

import random
import simpy

RANDOM_SEED = 42 # Semila aleatoria
NEW_CUSTOMERS = 5  # Número total de clientes
INTERVAL_CUSTOMERS = 10.0  # Genera nuevos clientes cada X segundos
MIN_PATIENCE = 3  # Paciencia mínima del cliente
MAX_PATIENCE = 5  # Paciencia máxima del cliente

def source(env, number, interval, counter):
    """
    La fuente genera clientes de manera
    aleatoria.
    """
    for i in range(number):
        c = customer(env, 'Cliente_%02d' % i, counter, time_in_bank=5.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """
    El cliente llega, es atentido y luego se va.
    """
    arrive = env.now
    print('%7.4f %s: Acá estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Espera al mostrador o se le agota la paciencia
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # El cliente llega al mostrador
            print('%7.4f %s: Esperó %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finalizó' % (env.now, name))

        else:
            # Al cliente se le agotó la paciencia
            print('%7.4f %s: RENEGÓ despues de %6.3f' % (env.now, name, wait))

# Prepara y empieza la simulación
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Empieza los procesos y ejecuta
counter = simpy.Resource(env, capacity=1)
customers_generator = source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
env.process(customers_generator)
env.run()

# **Carwash**
El ejemplo del Carwash es una simulación de un lavadero de carros con un número limitado de máquinas y un número de carros que llegan al lavadero para ser limpiados.

Cuando un carro llega al lavadero, éste requiere una máquina. Una vez que obtiene una, el lavado del carro se procesa y espera hasta que finaliza. Finalmente éste libera a la maquina y se va.

In [0]:

RANDOM_SEED = 42
NUM_MACHINES = 2  # Número de máquinas en el lavadero
WASHTIME = 5      # Minutos que le toma lavar un carro
T_INTER = 2       # Crea un carro cada ~7 minutos
SIM_TIME = 20     # Tiempo de simulación en minutos

class Carwash(object):
    """
    Un lavadero tiene un número limitado de máquinas NUM_MACHINES
    conas las que puede limpiar carros en paralelo.
    
    Los carros tienen que solicitar una de las máquinas. Cuando
    obtenga una, éstos pueden iniciar el proceso de lavado y esperar
    a que finalice (lo cual toma WASHTIME minutos).
    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """
        El proceso de lavado. Toma un proceso 'car' e
        intenta limpiarlo.
        """
        yield self.env.timeout(WASHTIME)
        print("El lavadero elimino el %d%% de la suciedad del %s." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """
    El proceso del carro (cada carro tiene un 'name') llega al
    lavadero ('cw') y solicita una maquina de limpiar.
    
    Luego empieza el proceso de lavado, espera a que finalice y
    se va para no volver nunca más.
    """
    print('%s llega al lavadero a los %.2f minutos.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra al lavadero a los %.2f minutos.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s sale del lavadero a los %.2f minutos.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """
    Crea un lavadero, un número inicial de carros y sigue
    creando carros durante aproximadamente 't_inter' minutos.
    """
    # Crea el lavadero
    carwash = Carwash(env, num_machines, washtime)

    # Crea 4 carros iniciales
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Crea más carros mientras la simulación se está ejecutando
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))

# Prepara e inicia la simulación
print('Carwash')
random.seed(RANDOM_SEED)

# Crea un ambiente e inicia el proceso de lavado
env = simpy.Environment()
carwash = setup(env, NUM_MACHINES, WASHTIME, T_INTER)
env.process(carwash)

# Ejecuta!
env.run(until=SIM_TIME)

# **Machine Shop**
Un taller tiene n máquinas idénticas. Un flujo de trabajo (suficiente para mantener a las máquinas ocupadas) llega. Cada máquina se rompe de manera periodica. Las reparaciones son hechas por un reparador. El reparador también tiene otras tareas menos importantes que hacer. Las máquinas rotas tienen prioridad sobre éstas tareas. El reparador continue con éstas una vez la máquina está reparada. El taller funciona de manera continua

In [0]:
RANDOM_SEED = 42
PT_MEAN = 10.0         # Tiempo promedio de procesamiento en minutos
PT_SIGMA = 2.0         # Sigma del tiempo de procesamiento
MTTF = 300.0           # Tiempo medio para fallar en minutos
BREAK_MEAN = 1 / MTTF  # Parametro para la distribución exponencial
REPAIR_TIME = 30.0     # Tiempo que toma reparar una máquina
JOB_DURATION = 30.0    # Duración de los otros trabajos en minutos
NUM_MACHINES = 10      # Número de máquinas en el taller
WEEKS = 4              # Tiempo de simulación en semanas
SIM_TIME = WEEKS * 7 * 24 * 60  # Tiempo de simulación en minutos

def time_per_part():
    """
    Retorna el tiempo actual de procesamiento para
    una parte en concreto.
    """
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """
    Retorna el tiempo de la siguiente falla
    en una máquina
    """
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """
    Una máquina produce partes y se daña de vez en cuando.
    
    Si se rompe, require un reparador y continua la producción luego
    de ser reparada.
    
    Una máquina tiene un nombre y número de partes hechas.
    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Empieza los procesos de 'working' y 'break_machine'
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """
        Produce partes mientras la simulación está corriendo.
        
        Mientras hace una parte, la máquina puede romperse
        multipples veces. Se solicita al reparador cuando esto
        sucede.
        """
        while True:
            # Comienza a hacer una parte
            done_in = time_per_part()
            while done_in:
                try:
                    # Trabajando en la parte
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Se pone en 0 para salir del ciclo.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # Cuanto tiempo falta?

                    # Solicita un reparador el cual dejara de lado sus otros trabajos
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """
        Daña la máquina de vez en cuando
        """
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Solo daña la máquina si actualmente está funcionando.
                self.process.interrupt()


def other_jobs(env, repairman):
    """
    Los otros trabajos no importantes del
    reparador
    """
    while True:
        # Empieza un nuevo trabajo
        done_in = JOB_DURATION
        while done_in:
            # Reintenta el trabajo hasta que esté hecho.
            # La prioridad es más bajo que la de la reparación de las máquinas.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start

# Prepara y empieza la simulación
print('Machine shop')
random.seed(RANDOM_SEED)

# Crea un ambiente y empiza el proceso de preparación
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Ejecuta!
env.run(until=SIM_TIME)

# Analisis/Resultados
print('Resultados del taller después de %s semanas:' % WEEKS)
for machine in machines:
    print('La %s hizo %d partes.' % (machine.name, machine.parts_made))
    

# **Movie Renege**
Este ejemplo modela un teatro de cine con un mostrador para la venta de tiquetes para tres peliculas (solo para el siguiente show). Las personas llegan en tiempo aleatorios e intentan comprar un número aleatorio de tiquetes (1-6) para una pelicula aleatoria. Cuando una pelicula está agotada, todas las personas que esperaban para comprar un tiquete para esa pelicula, reniegan (dejan la cola).

In [0]:
import collections
RANDOM_SEED = 42
TICKETS = 50  # Número de tiquetes por película
SIM_TIME = 120  # Simular hasta

def moviegoer(env, movie, num_tickets, theater):
    """
    Un cinéfilo intenta compra un número de tiquetes ('num_tickets')
    para una cierta 'movie' en un 'theater'.
    
    Si la pelicula se llena, el cinéfilo deja el teatro. Si el
    cinéfilo llega al mostrador, intenta comprar los tiquetes. Si
    no hay suficiente cantidad de tiquetes, el cinéfilo discute
    y se va.
    
    Si queda menos de un tiquete libre después de que el cinéfilo
    compre sus tiquetes, el evento 'sold out' se dispara causando
    que el resto de cinéfilos se vayan.
    """
    with theater.counter.request() as my_turn:
        # Espera su turno o hasta que la película se lleno
        result = yield my_turn | theater.sold_out[movie]

        # Valida si la pelicula se lleno
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Valida si hay suficientes tiquetes
        if theater.available[movie] < num_tickets:
            # El cinéfilo abandona después de una discusión
            yield env.timeout(0.5)
            env.exit()

        # Compra tiquetes
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Dispara el evento 'sould out'
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """
    Crea un nuevo cinéfilo hasta que el tiempo de
    simulación acabe
    """
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')

# Prepara y comienza la simulación
print('Movie Renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Crea el teatro de cine
counter = simpy.Resource(env, capacity=1)
movies = ['El Depredador', 'La Monja', 'Pesadilla Siniestra']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)


# Inicia el proceso y ejecuta
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analisis/Resultados
for movie in movies:
    if theater.sold_out[movie]:
        print('La pelicula "%s" se agotó %.1f minutos después de que el mostrador abriera.' % (movie, theater.when_sold_out[movie]))
        print('  Cantidad de personas que se fueron cuando se agotó la pelicula: %s' %
              theater.num_renegers[movie])

# **Gas Station Refueling**
Una estación de gas tiene un número limitado de surtidores que comparten una reserva común de combustible. Los carros llegan aleatoriamente a la estación de gas, solicitan uno de los surtidores y empiezan a rellenar de la reserva.

Un proceso de control de la estación de gas observa el nivel de combustible de la estación y llama a un carrotanque para rellenar en caso de que el nivel de la estación caiga por debajo de un umbral.

In [0]:
import itertools

RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # Litros
THRESHOLD = 10             # Umbral para llamar al carrotange (%)
FUEL_TANK_SIZE = 50        # Litros
FUEL_TANK_LEVEL = [5, 25]  # niveles min/max de los carrotanques (litros)
REFUELING_SPEED = 2        # litros por segundo
TANK_TRUCK_TIME = 300      # Segundos que se tarda en llegar el carrotanque
T_INTER = [30, 300]        # Crea un car cada [min, max] segundos
SIM_TIME = 1000            # Tiempo de simulación en segundos


def car(name, env, gas_station, fuel_pump):
    """
    Un carro llega a una estación de gas para repostar.
    
    Este requiere uno de los surtidores de la estación de gas
    e intenta obtener una cantidad deseada de gas de este.
    Si la reserva de la estación está agotada, el carro
    tendrá que esperar hasta que el carrotanque llegue.
    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llegando a la estación de gas a los %.1f segundos' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Solicita uno de los surtidores
        yield req

        # Obtiene la cantidad requerida de combustible
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # El proceso de repostar toma su tiempo
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s terminó de repostar en %.1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """
    Periodicamente revisa el nivel del 'fuel_pump' y llama al
    carrotanque si el nivel cae por debajo de un umbral.
    """
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # Necesitamos llamar a un carro tanque ahora!
            print('Llamando el carrotanque a los %.1f segundos' % env.now)
            # Esperamos a que llegue el carro tanque y rellene el deposito
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Revisa cada 10 segundos

def tank_truck(env, fuel_pump):
    """
    Llega a la estación de gas después de un cierto retraso
    y la rellena.
    """
    yield env.timeout(TANK_TRUCK_TIME)
    print('Carrotanque llegando a los %.1f segundos' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Carrotanque rellenando %.1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """
    Genera un nuevo carro que llega a la estación de gas
    """
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Carro %d' % i, env, gas_station, fuel_pump))


# Prepara e inicia la simulación
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Crea el ambiente e inicia los procesos
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Ejecuta!
env.run(until=SIM_TIME)